# Generate the agent
This notebook latter we will pass it into the `base agents.py`

In [4]:
#Imports
import os
import json

#Typing
from typing import Dict, List, Any, Optional, TypedDict
from IPython.display import Image, display

#LangGraph/LangChain
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI  # or any other LLM provider
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

# Prompts
#Load env files
from dotenv import load_dotenv
load_dotenv()




True

## Systems Prompts


In [ ]:
# Main Agent Prompts (largely unchanged since it's the orchestrator)
MAIN_AGENT_PROMPT = """You are an AI orchestrator for a story processing system. 
Analyze the user's request and the provided text to determine:
1. Is this an Aesop's fable? (yes/no)
2. What action should be taken? (analyze/retell/expand/modernize/create_new)
3. Any specific requirements? (style, moral, characters)

Respond in JSON format with keys: is_aesop, action, requirements"""

# Aesop Tool Prompts - Enhanced Analysis
ANALYZE_FABLE_PROMPT = """You are an expert in Aesop's fables and modern storytelling. Analyze the given fable and provide:
1. The core moral/lesson (what universal truth is it teaching?)
2. The conflict pattern (what fundamental challenge/dilemma does it present?)
3. Character archetypes and their essential traits (what roles do they serve?)
4. The narrative structure (setup, challenge, resolution)
5. What makes this moral relevant today

Format your response as JSON with these keys: moral, conflict_pattern, characters, structure, modern_relevance"""

# Enhanced Brainstorming for Modern Micro-Fables
BRAINSTORM_STORY_PROMPT = """You are a creative storyteller specializing in ultra-short modern fables for social media.
Based on the analysis provided, brainstorm ideas for a 100-110 word modern fable:

1. Fresh animal substitutions: Replace the original animals with unexpected species that maintain the same character traits but feel more surprising (consider unusual animals from diverse ecosystems)

2. Innovative settings: Suggest 2-3 unique settings beyond traditional forests (coral reefs, urban environments, cosmic settings, etc.)

3. Modern context: How could the core conflict be reframed in a contemporary or unexpected context while preserving the moral?

4. Implicit teaching approach: How to convey the moral without explicitly stating it, followed by a memorable 5-10 word takeaway phrase

Format your response as JSON with these keys: animal_substitutions, settings, modern_context, implicit_teaching, takeaway_phrase"""

# Enhanced Story Generation for TikTok-Style Fables
GENERATE_STORY_AESOP_PROMPT = """You are a master of micro-storytelling creating modern Aesop fables for the TikTok generation.
Create a compelling 120-130 word fable that:

1. Uses the suggested animal substitutions and innovative setting
2. Presents a complete narrative arc (setup, conflict, resolution) with extreme efficiency
3. Teaches the moral implicitly through the story
4. Uses vivid, sensory language that creates mental images
5. Ends with the suggested 5-10 word takeaway phrase (not an explicit "the moral is...")

STRICT CONSTRAINTS:
- Exactly 110-100 words for the main story
- Takeaway phrase should be 5-10 words, positioned at the end
- No explicit statement of "the moral is..." or "this teaches us..." in the story
- Every word must serve multiple purposes (character, plot, and theme)

Your task is to distill ancient wisdom into a shareable modern micro-fable."""

# Enhanced Output Formatting
FORMAT_OUTPUT_PROMPT = """Format this modern Aesop fable for maximum impact on social platforms:

1. Present the story as a complete micro-fable (exactly as generated, preserving the 100-110 word count)
2. Set the takeaway phrase on its own line at the end, styled for emphasis
3. Include a brief note about the original fable it's based on
4. Mention one interesting insight about how this modern version preserves the timeless wisdom

Keep the entire output concise and visually scannable - perfect for a quick digital read."""

## Create metadata class

In [6]:
import time
import json
import functools

from datetime import datetime
import tiktoken  # For token counting

# Global metadata store
metadata = {
    "session_start": time.time(),
    "session_id": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "current_story": None,
    "stories": {},
    "total_tokens": 0,
    "total_cost": 0,
    "total_time": 0
}

def track_node(node_name, tool_name="main"):
    """Decorator to track execution time of nodes"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(state):
            # Get current story ID or create one
            story_id = metadata.get("current_story")
            if not story_id:
                story_id = f"story_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                metadata["current_story"] = story_id
                metadata["stories"][story_id] = {
                    "start_time": time.time(),
                    "nodes": {},
                    "tools": {},
                    "llm_calls": [],
                    "total_tokens": 0,
                    "total_cost": 0
                }
            
            # Initialize node data if needed
            story = metadata["stories"][story_id]
            if node_name not in story["nodes"]:
                story["nodes"][node_name] = {
                    "calls": 0,
                    "total_time": 0,
                    "tokens": 0
                }
            
            # Start timing
            start_time = time.time()
            
            # Call the function
            try:
                result = func(state)
                # Record timing data
                end_time = time.time()
                duration = end_time - start_time
                
                # Update metrics
                story["nodes"][node_name]["calls"] += 1
                story["nodes"][node_name]["total_time"] += duration
                
                print(f"Node {node_name} executed in {duration:.2f} seconds")
                
                return result
            except Exception as e:
                # Record error but re-raise
                end_time = time.time()
                duration = end_time - start_time
                
                print(f"Error in {node_name}: {str(e)}")
                story["nodes"][node_name]["calls"] += 1
                story["nodes"][node_name]["total_time"] += duration
                story["nodes"][node_name]["errors"] = story["nodes"][node_name].get("errors", 0) + 1
                
                raise
        return wrapper
    return decorator

def track_llm_call(node_name, tool_name, model, system_prompt, user_prompt, response_text):
    """Track an LLM API call"""
    story_id = metadata.get("current_story")
    if not story_id:
        return
    
    # Simple token estimation (very rough)
    input_tokens = (len(system_prompt) + len(user_prompt)) // 4  # ~4 chars per token
    output_tokens = len(response_text) // 4
    
    # Cost estimation (very rough)
    if model == "gpt-4.1-mini":
        input_cost = (input_tokens / 1000) * 0.00015
        output_cost = (output_tokens / 1000) * 0.00060
    else:
        input_cost = (input_tokens / 1000) * 0.00010
        output_cost = (output_tokens / 1000) * 0.00030
    
    total_cost = input_cost + output_cost
    
    # Record the call
    call_data = {
        "timestamp": time.time(),
        "node": node_name,
        "tool": tool_name,
        "model": model,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": input_tokens + output_tokens,
        "total_cost": total_cost
    }
    
    # Update story
    story = metadata["stories"][story_id]
    story["llm_calls"].append(call_data)
    story["total_tokens"] += input_tokens + output_tokens
    story["total_cost"] += total_cost
    
    # Update node
    if node_name in story["nodes"]:
        story["nodes"][node_name]["tokens"] += input_tokens + output_tokens
    
    # Update global
    metadata["total_tokens"] += input_tokens + output_tokens
    metadata["total_cost"] += total_cost
    
    print(f"LLM call in {node_name}: {input_tokens + output_tokens} tokens, ${total_cost:.4f}")

def finish_story(output_text):
    """Finish tracking the current story"""
    story_id = metadata.get("current_story")
    if not story_id:
        return
    
    story = metadata["stories"][story_id]
    story["end_time"] = time.time()
    story["duration"] = story["end_time"] - story["start_time"]
    story["output_length"] = len(output_text)
    
    # Calculate summary stats
    total_time = sum(node["total_time"] for node in story["nodes"].values())
    total_calls = sum(node["calls"] for node in story["nodes"].values())
    
    print("\n=== STORY METRICS ===")
    print(f"Total execution time: {story['duration']:.2f} seconds")
    print(f"Total tokens: {story['total_tokens']} tokens")
    print(f"Estimated cost: ${story['total_cost']:.4f}")
    print(f"Number of nodes executed: {len(story['nodes'])}")
    print(f"Number of LLM calls: {len(story['llm_calls'])}")
    
    # Reset current story
    metadata["current_story"] = None
    metadata["total_time"] += story["duration"]
    
    # Export metadata
    with open(f"story_metrics_{story_id}.json", "w") as f:
        json.dump(story, f, indent=2, default=str)
    
    print(f"Metrics saved to story_metrics_{story_id}.json")
    
    return story

## Step 1:
Import OPENAI as a global variable and define it.

In [7]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Call the api keys from the .env file
llm_openai_41_mini = ChatOpenAI(model="gpt-4.1-mini")

## Step 2:
Define the the states of the main graph and the subgraphs

In [8]:
class MainState(TypedDict):
    messages: List[Dict[str, Any]]  # User messages
    current_fable: str              # Input fable text
    tool_to_call: str               # Which tool subgraph to use
    processing_request: Dict        # Request info for the tool
    tool_output: Dict               # Output from selected tool
    final_story: str                # Final output after post-processing

# Cell 3: Define Aesop State (for subgraph)
class AesopState(TypedDict):
    original_fable: str             # Original input text
    analysis: Dict                  # Analysis of the fable (moral, characters, etc)
    brainstorm: Dict                # Ideas for the story creation/modification
    generated_story: str            # The final story created by this tool

## Step 3:
Create the story router

In [16]:

# Cell 4: Main Agent for the main graph
@track_node("main_agent", "main")
def main_agent(state: MainState) -> Dict[str, Any]:
    """
    Main orchestrator that analyzes input and decides which tool to use
    """
    print("\n=== Main Agent ===")
    current_fable = state.get("current_fable", "")
    messages = state.get("messages", [])
    
    user_message = messages[-1].get("content", "") if messages else ""
    print(f"Processing request: {user_message}")
    print(f"Current fable length: {len(current_fable)} characters")
    
    is_aesop = True
    
    if is_aesop:
        processing_request = {
            "user_intent": user_message,
            "fable_text": current_fable
        }
        
        result = {
            "processing_request": processing_request,
            "tool_to_call": "aesop_tool"
        }
        print(f"Selecting tool: aesop_tool")
        return result
    else:
        # Future expansion for other tools
        return {
            "processing_request": {},
            "tool_to_call": "generic_tool" 
        }

# Cell 5: Tool Router
def tool_router(state: MainState) -> Dict[str, Any]:
    """
    Routes to the appropriate tool subgraph
    """
    print("\n=== Tool Router ===")
    tool_name = state.get("tool_to_call", "")
    processing_request = state.get("processing_request", {})
    
    print(f"Routing to tool: {tool_name}")
    
    if tool_name == "aesop_tool":
        # Call the Aesop subgraph
        aesop_result = aesop_subgraph(processing_request)
        print(f"Received result from Aesop tool")
        return {"tool_output": aesop_result}
    else:
        # Future: add more tool subgraphs
        return {"tool_output": {"error": "Tool not found"}}

@track_node("generate_output", "main")
def generate_output(state: MainState) -> Dict[str, Any]:
    """
    Formats the final output based on the tool results
    Creates a two-part story with strict word count limits
    """
    print("\n=== Generate Output ===")
    tool_output = state.get("tool_output", {})
    tool_name = state.get("tool_to_call", "")
    
    if "error" in tool_output:
        final_story = f"Error: {tool_output['error']}"
    else:
        if tool_name == "aesop_tool":
            # Get story components from the tool output
            generated_story = tool_output.get("generated_story", "")
            analysis = tool_output.get("analysis", {})
            brainstorm = tool_output.get("brainstorm", {})
            
            # Track the original story length for metadata
            original_word_count = len(generated_story.split())
            print(f"Original story word count: {original_word_count}")
            
            # Create a prompt to split and enhance the story with strict word counts
            system_prompt = """You are a master storyteller specializing in micro-fables for social media.
            Take this fable and transform it into a two-part story with STRICT word count limits:
            
            PART 1 (First Post):
            - EXACTLY 80-90 WORDS MAXIMUM
            - Contains the setup, characters, and builds tension
            - Ends at a compelling moment that makes readers curious for the conclusion
            - Should be clearly labeled as "PART 1"
            
            PART 2 (Second Post):
            - EXACTLY 80-90 WORDS MAXIMUM (including the recap)
            - Begins with a brief 1-sentence recap of Part 1, and explicitly add at the begginig In Part 1
            - Contains the resolution and moral/takeaway
            - Ends with a memorable phrase that captures the moral
            - Should be clearly labeled as "PART 2"
            
            The word counts are STRICT REQUIREMENTS - do not exceed 80 words for each part.
            Count your words carefully before finalizing each part.
            """
            
            user_prompt = f"""Story: {generated_story}
            
            Analysis: {json.dumps(analysis, indent=2)}
            
            Split this into a two-part story as described, with exactly 80 words maximum for each part.
            Ensure Part 2 begins with a brief recap so viewers can understand the conclusion even if they missed Part 1.
            """
            
            # Call LLM to reformat the story
            response = llm_openai_41_mini.invoke([
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_prompt)
            ])
            
            # Track LLM call
            track_llm_call(
                node_name="generate_output",
                tool_name="main",
                model="gpt-4.1-mini",
                system_prompt=system_prompt,
                user_prompt=user_prompt,
                response_text=response.content
            )
            
            # The final reformatted story
            final_story = response.content
            
            # Track final word count for metadata
            final_word_count = len(final_story.split())
            print(f"Final story word count: {final_word_count}")
            
            # Verify the word counts of each part (for validation and debugging)
            parts = final_story.split("PART 2")
            if len(parts) > 1:
                part1 = parts[0].replace("PART 1", "").strip()
                part2 = "PART 2" + parts[1].strip()
                
                part1_words = len(part1.split())
                part2_words = len(part2.split())
                
                print(f"Part 1 word count: {part1_words}")
                print(f"Part 2 word count: {part2_words}")
                
                # Add to metadata
                if "current_story" in metadata and metadata["current_story"] in metadata["stories"]:
                    story_id = metadata["current_story"]
                    if "transformations" not in metadata["stories"][story_id]:
                        metadata["stories"][story_id]["transformations"] = {}
                    
                    metadata["stories"][story_id]["transformations"]["story_split"] = {
                        "original_word_count": original_word_count,
                        "final_word_count": final_word_count,
                        "part1_word_count": part1_words,
                        "part2_word_count": part2_words,
                        "transformation_type": "two-part-fixed-length",
                        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    
                    print("Added transformation metadata")
        else:
            # Future: handle other tool outputs with different formatting strategies
            final_story = str(tool_output)
    
    print(f"Final story generated (length: {len(final_story)} characters)")
    return {"final_story": final_story}

## Step 4:
Create the tools-subgraphs, currently we have:
* Aesop tool-subgraph

In [ ]:
# Node 1: Analyze Fable with tracking
@track_node("analyze_fable", "aesop_tool")
def analyze_fable(state: AesopState) -> Dict[str, Any]:
    """
    Analyzes the fable structure, characters, and moral
    """
    print("\n== Aesop Subgraph: Analyze Fable ==")
    original_fable = state.get("original_fable", "")
    print(f"Analyzing fable (length: {len(original_fable)} characters)")
    
    # Use LLM to analyze the fable with prompt from prompts.py
    system_prompt = ANALYZE_FABLE_PROMPT
    
    user_prompt = f"Analyze this fable:\n\n{original_fable}"
    
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt)
    ])
    
    # Track the LLM call
    track_llm_call(
        node_name="analyze_fable",
        tool_name="aesop_tool",
        model="gpt-4.1-mini",
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        response_text=response.content
    )
    
    try:
        analysis = json.loads(response.content)
    except:
        # Fallback if JSON parsing fails
        print("JSON parsing failed, using content as analysis")
        analysis = {
            "moral": response.content,
            "characters": [],
            "structure": {},
            "symbols": []
        }
    
    print(f"Analysis complete: identified moral '{analysis.get('moral', '')[:50]}...'")
    return {"analysis": analysis}

@track_node("brainstorm_story", "aesop_tool")
# Node 2: Brainstorm Story
def brainstorm_story(state: AesopState) -> Dict[str, Any]:
    """
    Brainstorms ideas for story creation based on analysis
    """
    print("\n== Aesop Subgraph: Brainstorm Story ==")
    original_fable = state.get("original_fable", "")
    analysis = state.get("analysis", {})
    
    print(f"Brainstorming based on analysis of moral: {analysis.get('moral', '')[:50]}...")
    
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=BRAINSTORM_STORY_PROMPT),
        HumanMessage(content=f"Original fable: {original_fable}\n\nAnalysis: {json.dumps(analysis, indent=2)}")
    ])
    
    try:
        brainstorm = json.loads(response.content)
    except:
        print("JSON parsing failed, using content as brainstorm")
        brainstorm = {
            "moral_approaches": response.content,
            "variations": [],
            "character_ideas": [],
            "imagery": []
        }
    
    print(f"Brainstorming complete: generated {len(brainstorm.keys())} idea categories")
    return {"brainstorm": brainstorm}


@track_node("generate_story_aessop", "aesop_tool")
# Node 3: Generate Story
def generate_story_aessop(state: AesopState) -> Dict[str, Any]:
    """
    Generates the final story based on analysis and brainstorming
    """
    print("\n== Aesop Subgraph: Generate Story ==")
    original_fable = state.get("original_fable", "")
    analysis = state.get("analysis", {})
    brainstorm = state.get("brainstorm", {})
    
    print(f"Generating story based on analysis and brainstorming")
    
    response = llm_openai_41_mini.invoke([
        SystemMessage(content=GENERATE_STORY_AESOP_PROMPT),
        HumanMessage(content=f"""
        Original fable: {original_fable}
        
        Analysis: {json.dumps(analysis, indent=2)}
        
        Brainstorming: {json.dumps(brainstorm, indent=2)}
        
        Create a refined version of this fable.
        """)
    ])
    
    generated_story = response.content
    print(f"Story generated (length: {len(generated_story)} characters)")
    
    return {"generated_story": generated_story}

# Cell 8: BUILD THE AESOP SUBGRAPH
def build_aesop_subgraph():
    """
    Builds the Aesop tool subgraph
    """
    builder = StateGraph(AesopState)
    
    # Add nodes
    builder.add_node("analyze_fable", analyze_fable)
    builder.add_node("brainstorm_story", brainstorm_story)
    builder.add_node("generate_story_aessop", generate_story_aessop)
    
    # Add edges
    builder.add_edge(START, "analyze_fable")
    builder.add_edge("analyze_fable", "brainstorm_story")
    builder.add_edge("brainstorm_story", "generate_story_aessop")
    builder.add_edge("generate_story_aessop", END)
    
    # Compile
    return builder.compile()

# Cell 9: AESOP SUBGRAPH WRAPPER
def aesop_subgraph(processing_request: Dict) -> Dict:
    """
    Wrapper function that runs the Aesop subgraph
    """
    print("\n=== Running Aesop Subgraph ===")
    
    # Create initial state for Aesop subgraph
    initial_state = {
        "original_fable": processing_request.get("fable_text", ""),
        "analysis": {},
        "brainstorm": {},
        "generated_story": ""
    }
    
    # Build and run the subgraph
    aesop_graph = build_aesop_subgraph()
    result = aesop_graph.invoke(initial_state)
    
    # Return the enriched state
    return {
        "analysis": result["analysis"],
        "brainstorm": result["brainstorm"],
        "generated_story": result["generated_story"]
    }

## Step 5:
Build the main graph

In [11]:
# Cell 10: BUILD THE MAIN GRAPH
def build_main_graph():
    """
    Builds the main orchestrator graph
    """
    builder = StateGraph(MainState)
    
    # Add nodes
    builder.add_node("main_agent", main_agent)
    builder.add_node("tool_router", tool_router)
    builder.add_node("generate_output", generate_output)
    
    # Add edges
    builder.add_edge(START, "main_agent")
    builder.add_edge("main_agent", "tool_router")
    builder.add_edge("tool_router", "generate_output")
    builder.add_edge("generate_output", END)
    
    # Compile
    return builder.compile()

In [12]:
# Cell 11: TEST SYSTEM
def test_system():
    # Build the main graph
    main_graph = build_main_graph()
    print("Graph built successfully!")
    
    # Test with a sample Aesop fable
    test_fable = """
    The Fox and the Grapes
    
    A hungry Fox saw some fine bunches of Grapes hanging from a vine that was trained along a high trellis, and did his best to reach them by jumping as high as he could into the air. But it was all in vain, for they were just out of reach: so he gave up trying, and walked away with an air of dignity and unconcern, remarking, "I thought those Grapes were ripe, but I see now they are quite sour."
    """
    
    initial_state = {
        "messages": [{"role": "user", "content": "Analyze and enhance this fable"}],
        "current_fable": test_fable,
        "tool_to_call": "",
        "processing_request": {},
        "tool_output": {},
        "final_story": ""
    }
    
    # Run the graph
    result = main_graph.invoke(initial_state)
    
    # Finish tracking this story
    story_stats = finish_story(result["final_story"])
    
    print("\n=== FINAL RESULT ===")
    print(result["final_story"])
    
    return result

## Step 6:
Test the system

In [17]:
# Cell to actually run the test
test_result = test_system()

Graph built successfully!

=== Main Agent ===
Processing request: Analyze and enhance this fable
Current fable length: 432 characters
Selecting tool: aesop_tool
Node main_agent executed in 0.00 seconds

=== Tool Router ===
Routing to tool: aesop_tool

=== Running Aesop Subgraph ===

== Aesop Subgraph: Analyze Fable ==
Analyzing fable (length: 432 characters)
LLM call in analyze_fable: 604 tokens, $0.0003
Analysis complete: identified moral 'It is easy to despise what you cannot have; people...'
Node analyze_fable executed in 4.45 seconds

== Aesop Subgraph: Brainstorm Story ==
Brainstorming based on analysis of moral: It is easy to despise what you cannot have; people...
Brainstorming complete: generated 5 idea categories
Node brainstorm_story executed in 3.38 seconds

== Aesop Subgraph: Generate Story ==
Generating story based on analysis and brainstorming
Story generated (length: 710 characters)
Node generate_story executed in 2.66 seconds
Received result from Aesop tool

=== Generat